# Create a new private Collection

The script in this notebook creates a new private Collection.

In order to use this script, you must have a Curation API key (obtained from upper-righthand dropdown in the Data Portal UI after logging in).

#### <font color='#bc00b0'>Please fill in the required values:</font>

<font color='#bc00b0'>(Required) Provide the path to your api key file</font>

In [ ]:
api_key_file <- "path/to/api-key.txt"

### Import dependencies

In [ ]:
library("readr")
library("httr")
library("stringr")
library("rjson")

### Specify domain (and API url)

In [ ]:
domain_name <- "cellxgene.cziscience.com"
site_url <- str_interp("https://${domain_name}")
api_url_base <- str_interp("https://api.${domain_name}")

### Use API key to obtain a temporary access token

In [ ]:
api_key <- read_file(api_key_file)
access_token_path <- "/curation/v1/auth/token"
access_token_url <- str_interp("${api_url_base}${access_token_path}")
res <- POST(url=access_token_url, add_headers(`x-api-key`=api_key))
stop_for_status(res)
access_token <- content(res)$access_token

##### (optional, debug) verify status code of response

In [ ]:
print(res$status_code)

### Collection metadata (required)

In [ ]:
collection_form_metadata <- list(
  name = "Name",
  description = "A sample description",
  contact_name = "First Last",
  contact_email = "firstlast@email.com",
  links = list(
      list(
          link_name = "sample protocol link",
          link_url = "https://www.sample.link",
          link_type = "PROTOCOL"
      ),
      list(
          link_name = "sample lab website",
          link_url = "https://www.lab.website",
          link_type = "LAB_WEBSITE"
      )
  )
)

### Formulate request and create new private Collection

In [ ]:
collections_path <- "/curation/v1/collections"
collections_url <- str_interp("${api_url_base}${collections_path}")
bearer_token <- str_interp("Bearer ${access_token}")
print(toJSON(collection_form_metadata))
res <- POST(
    url=collections_url, body=toJSON(collection_form_metadata),
    add_headers(`Authorization`=bearer_token, `Content-Type`="application/json")
)
stop_for_status(res)
res_content <- content(res)
print("New private Collection uuid:")
print(res_content$collection_uuid)
print("New private Collection url:")
print(str_interp("${site_url}/collections/${res_content$collection_uuid}"))
